In [1]:
import cantera as ct
import numpy as np
import scipy as sp
import pandas as pd
from pint import UnitRegistry
import time

ureg = UnitRegistry()
Q = ureg.Quantity

# Introduction
## Purpose
The purpose of this code is as follows:
- Find equilibrium canditions via cantera
- Calculate Specific heat (without condensed species)
- Calculate thermo derivatives (for above calculations)

## Next Steps
- Calculate CEA rocket output properties: Isp, C*, etc.
- Calculate thermal conductivity, viscocity
- Calculate Bartz heat transfer coefficient

## Relevant Expressions & Variables
### Matrices
$a_{ij}$ is the stoichiometric coefficients matrix. Defined as the number of kilogram-moles of element $i$ per kilogram-mole species $j$ 

### Thermodynamic Properties
#### Specific Heat
Specific heat can be said to have a frozen and Equilibrium component. If $C_p = dH/dT$ where $H = nh$ where $n$ is moles and $h$ is molar enthalpy. Often $n$ is unchanging but if we have a reacting mixture then we get: $C_p = n\frac{dh}{dT} + \frac{dn}{dT}h$ 
This the first term is said to be the frozen specific heat as the $n$ is constant, while the second term is the reacting specific heat. As shown bellow
$$\begin{align}
c_{p,e}=c_{p,f} + c_{p,r}= \sum_{j=1}^{NS}n_jC_{p,j}^o+\sum_{j=1}^{NG} n_j\frac{H_j^o}{T}\left(\frac{\partial\ln n_j}{\partial \ln T}\right)_P+\sum_{j=NG+1}^{NS} \frac{H_j^o}{T}\left(\frac{\partial n_j}{\partial \ln T}\right)_P
\end{align}$$

$$\begin{align}
c_v \equiv c_p + \frac{\frac{PV}{T}\left(\frac{\partial \ln V}{\partial \ln T}\right)_p^2}{\left(\frac{\partial \ln V}{\partial \ln P}\right)_T}\tag{2.70}
\end{align}$$

#### Specific heat ratio
$$\begin{align}
\gamma \equiv \frac{c_p}{c_v}\tag{2.72}
\end{align}$$

$$\begin{align}
\gamma_s = \frac{\gamma}{\left(\frac{\partial \ln V}{\partial \ln P}\right)_T}\tag{2.73}
\end{align}$$

$$\begin{align}
a= \sqrt{nRT\gamma_s} \tag{2.74}
\end{align}$$

### Thermodyanamic Derivatives
#### Derivatives with Respect to Temperature
$$\begin{align}
\sum_{i=1}^\ell \sum_{j=1}^{NG} a_{kj} a_{ij} n_j \left( \frac{\partial \pi_i}{\partial \ln T} \right)_P 
&+ \sum_{j=NG+1}^{NS} a_{ij} \left( \frac{\partial n_j}{\partial \ln T} \right)_P \notag
\\ &+ \sum_{j=1}^{NG} a_{kj} n_j \left( \frac{\partial \ln n}{\partial \ln T} \right)_P 
= -\sum_{j=1}^{NG} a_{kj} n_j \frac{H_j^o}{RT} \qquad k = 1, \ldots, \ell
\end{align}$$

$$\begin{align}
\sum_{i=1}^\ell a_{ij} \left( \frac{\partial \pi_i}{\partial \ln T} \right)_P =-\frac{H_j^o}{RT}\qquad j=NG+1,\ldots,NS\tag{2}
\end{align}$$

$$\begin{align}
\sum_{i=1}^\ell \sum_{j=1}^{NG} a_{ij} n_j \left( \frac{\partial \pi_i}{\partial \ln T} \right)_P
=-\sum_{j=1}^{NG}\frac{n_jH_j^o}{RT}\tag{3}
\end{align}$$

#### Derivatives with Respect to Pressure
$$\begin{align}
\sum_{i=1}^\ell \sum_{j=1}^{NG} a_{kj} a_{ij} n_j \left( \frac{\partial \pi_i}{\partial \ln P} \right)_T
&+ \sum_{j=NG+1}^{NS} a_{ij} \left( \frac{\partial n_j}{\partial \ln P} \right)_T \notag
\\ &+ \sum_{j=1}^{NG} a_{kj} n_j \left( \frac{\partial \ln n}{\partial \ln P} \right)_T
= -\sum_{j=1}^{NG} a_{kj} n_j  \quad k = 1, \ldots, \ell\tag{4}
\end{align}$$

$$\begin{align}
\sum_{i=1}^\ell a_{ij} \left( \frac{\partial  \pi_i}{\partial \ln P} \right)_T 
=0\qquad j=NG+1,\ldots,NS\tag{5}
\end{align}$$

$$\begin{align}
\sum_{i=1}^\ell \sum_{j=1}^{NG} a_{ij} n_j \left( \frac{\partial \pi_i}{\partial \ln T} \right)_P
=\sum_{j=1}^{NG}n_j\tag{6}
\end{align}$$

#### Derivatives of volume
$$\begin{align}
\left( \frac{\partial  V}{\partial \ln T} \right)_P = 1 + \left( \frac{\partial  \ln n}{\partial \ln T} \right)_P\tag{2.50}
\end{align}$$

$$\begin{align}
\left( \frac{\partial  V}{\partial \ln P} \right)_T = -1 + \left( \frac{\partial  \ln n}{\partial \ln P} \right)_T\tag{2.51}
\end{align}$$

### Engine conditions
Initial estimate at throat pressure is given as
$$\begin{align}
\frac{P_{inf}}{P_t}=\left(\frac{\gamma_s+1}{2}\right)^{\gamma_s/(\gamma_s-1)}\tag{6.15}
\end{align}$$

### Characteristic velocity
from Sutton
$$\begin{align}
c^*=\frac{\sqrt{\gamma \frac{\bar R}{M}T}}{\gamma \sqrt{\left(\frac2{K+1}\right)^{\frac{\gamma+1}{\gamma-1}}}}\tag{3-32}
\end{align}$$
From which we can derive 
$$\begin{align}
c^*=\sqrt{\frac{RT}{\gamma M}}\left(\frac{2}{\gamma+1}\right)^{-\frac{\gamma+1}{2(\gamma-1)}}
\end{align}$$

### lagrangian multiplier 
$$\pi_i=-\gamma_i/RT$$ 
where $\gamma_i$ are the lagrangian multipliers such that that $\mu_j+\sum_{i=1}^\ell \lambda_ia_{ij}=0$

### Isentropic flow equation
$$\begin{align}
P_t = \frac{P_0}{\frac{\gamma+1}{2}^{\frac{\gamma}{\gamma-1}}} \tag{6.15}
\end{align}$$

https://www.grc.nasa.gov/www/k-12/airplane/isentrop.html


$$\begin{align}
\ln \frac{P_c}{P_e}=\frac{\ln\left(\frac{P_c}{P_t}\right)}{\frac{A_e}{A_t}+10.587\left(\ln\frac{A_e}{A_t}\right)^3+9.454\ln\frac{A_e}{A_t}}\qquad 1.0001 < \frac{A_e}{A_t} < 1.09\tag{6.20}
\end{align}$$

In [113]:
class engine_state:
    '''
    This class will define the properties of an rocket engine at the chamber, throat and exit 
    given the propellant, and conditions that engine is in.

    Parameters
    --------------------------------------------------
    oxidizer : string
        A string of a species name which is used to define a cantera solution representing the oxidizer.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    fuel : string
        Similar to oxidizer, this is a string of a species name which is used to define a cantera solution representing the fuel.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    of_ratio : float
        the mass ratio of oxidizer to fuel defined as mass_oxidizer / mass_fuel.
    pressure : float
        The pressure of the combustion chamber in units of Pascals (Pa).
    exit_condition : float
        The exit conditions of the engine. Depending on what 'exit parameter' is
        defined to be the exit condition can be defined in the following ways:
            'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
            'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
    exit_parameter : string
        A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
        or 'area ratio'. 
    temp_oxidizer : float
        A positive float value defining the temperature of the oxidizer in Kelvin (K). If the oxidizer is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
    temp_fuel : float
        Similar to above. positive float value defining the temperature of the fuel in Kelvin (K). If the fuel is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality

    Attributes:
    --------------------------------------------------
    Properties : Pandas Dataframe
    
    Methods
    --------------------------------------------------

    A class for the state of a rocket engine. Given the propellants and conditions of a rocket engine this 
    class will define the properties of an engine at
    '''

    def __init__(self, oxidizer, fuel, of_ratio, pressure, exit_condition, temp_oxidizer=None, temp_fuel=None, combustion_products=None, exit_parameter="pressure"):
        '''
        Initializes instance of class. See class description for details. 
        '''
        #TODO: Update File handling, to support more propellants, enable user defining products, if undefined searching for propellants ourself        
        
        # initializes propellants
        cantera_input = 'RoCT/propellants.yaml'
        self.fuel = ct.Solution(cantera_input, fuel) 
        self.fuel.TP = temp_fuel, pressure
        self.oxidizer = ct.Solution(cantera_input, oxidizer)
        self.oxidizer.TP = temp_oxidizer, pressure
        self.of_ratio = of_ratio

        # initializes combustion products
        if combustion_products == None:
            self._products = [S for S in ct.Species.list_from_file('nasa_gas.yaml')]
        else:
            species = {S.name: S for S in ct.Species.list_from_file('nasa_gas.yaml')}
            self._products = ct.Solution(thermo='ideal-gas', species=[species[S] for S in combustion_products])
        
        self.__chamber_properties(self._products)
        self.__throat_properties(self._products)
        self.__exit_properties(self._products, exit_condition, exit_parameter=exit_parameter)

        self.engine_state = pd.DataFrame([self.chamber, self.throat, self.exit], index=["chamber", "throat", "exit"])

    def __call__(self):
        return self.engine_state
    
    def __str__(self):
        return self.engine_state.to_string()

    def __get_thermo_derivatives(self, mixture):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species this method will find thermodynamic derivatives.
        These derivatives can then be used to calculate thermodynamic properties, or create interpolation/extrapolation of engine properties 

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.
        The equations used are 2.50, 2.51, 2.56, 2.57, 2.58, 2.64, 2.65, 2.66 

        TODO: add condensed species to implementation
        TODO: explore interpolation of engine properties using derivatives
        
        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        
        Returns
        --------------------------------------------------
        Derivatives : dictionary:
            A dictionary consisting of the derivatives this function calculates. The keys and descritions of said dictionaries are as follows
        
            dpi_dlnT_P : list
                A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
                and lambda is the langrangian multiplier
            dlnn_dlnT_P : float
                The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas
            dpi_dlnP_T : list
                A list of the derivative of pi with respect to ln(P) at constant temperature for each element.
            dlnn_dlnP_T : float
                The derivative of ln(n) with respect to ln(P) at constant tempearture.
            dlnV_dlnT_P : float
                The derivative of ln(V) with respect to ln(T) at constant pressure.
            dlnV_dlnP_T : float
                The derivative of ln(V) with respect to ln(P) at constant tempearture.
        '''
        
        # Initializes a_ij
        a_ij = np.zeros((mixture.n_elements, mixture.n_species))
        for i, element in enumerate(mixture.element_names): 
            for j, species in enumerate(mixture.species_names):
                a_ij[i,j] = mixture.n_atoms(species, element)

        # Defines the number of moles of each species in the micture
        moles = mixture.X * (1/ mixture.mean_molecular_weight)

        # Initializing Solution Matrices table 2.3 and 2.4 in RP-1311
        num_variables = 2 * mixture.n_elements + 2
        coeff_matrix = np.zeros((num_variables, num_variables))
        right_hand_side = np.zeros(num_variables)

        # Coefficients for equation 2.56 TODO: add terms for condensed species
        for k in range(mixture.n_elements):
            for i in range(mixture.n_elements):
                coeff_matrix[k,i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[k, mixture.n_elements] = np.sum(a_ij[k,:] * moles)
            right_hand_side[k] = -np.sum(a_ij[k,:] * moles * mixture.standard_enthalpies_RT)

        # TODO add equation 2.57 (it is for condensed species)

        # Coefficients for equation 2.58 
        for i in range(mixture.n_elements):
            coeff_matrix[mixture.n_elements, i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[mixture.n_elements] = -np.sum(moles * mixture.standard_enthalpies_RT)

        # Coefficients for equation 2.64 TODO: add terms for condensed species
        for k in range(mixture.n_elements):
            for i in range(mixture.n_elements):
                coeff_matrix[mixture.n_elements+1+k,mixture.n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[mixture.n_elements+1+k, 2*mixture.n_elements+1] = np.sum(a_ij[k,:] * moles)
        right_hand_side[mixture.n_elements+1+k] = np.sum(a_ij[k,:] * moles)

        # TODO add equation 2.65 (it is for condensed species)
        

        # Coefficeints for equation 2.66
        for i in range(mixture.n_elements):
            coeff_matrix[2*mixture.n_elements+1, mixture.n_elements+1+i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[2*mixture.n_elements+1] = np.sum(moles)

        # Solve for the derivatives define them based off table 2.3, 2.4 and equation 2.50 and 2.51 
        derivs = np.linalg.solve(coeff_matrix, right_hand_side)
        derivatives = { "dpi_dlnT_P"    : derivs[0 : mixture.n_elements], 
                        "dlnn_dlnT_P"   : derivs[mixture.n_elements], 
                        "dpi_dlnP_T"    : derivs[mixture.n_elements + 1: 2 * mixture.n_elements + 1],
                        "dlnn_dlnP_T"   : derivs[2 * mixture.n_elements + 1], 
                        "dlnV_dlnT_P"   : 1 + derivs[mixture.n_elements], 
                        "dlnV_dlnP_T"   : -1 + derivs[2 * mixture.n_elements + 1]}

        return derivatives

    def __get_thermo_properties(self, mixture, dpi_dlnT_P, dlnn_dlnT_P, dlnV_dlnT_P, dlnV_dlnP_T):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic derivatives of said mixture, this function will find 
        the thermodynamic properties of the mixture.

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        TODO: add condensed species to implementation
        TODO: add capability to solve for thermal conductivity, viscocity, and prandtl number  

        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        dpi_dlnT_P : list
            A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
            and lambda is the langrangian multiplier
        dlnn_dlnT_P : float
            The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas.
        dlnV_dlnT_P : float
            The derivative of ln(V) with respect to ln(T) at constant pressure.
        dlnV_dlnP_T : float
            The derivative of ln(V) with respect to ln(P) at constant tempearture.
            
        Returns
        --------------------------------------------------
        properties : dictionaries
            A dictionary consisting of the properties this function calculates. The keys and descritions of said dictionaries are as follows

            Pressure : float
            Temperature : float
            density : float
            specific_volume : float
            enthalpy : float
            internal_energy : float
            gibbs : float
            entropy : float
            molar_mass : float
            c_p : float
                the specific heat at constant pressure
            c_v : float
                the specific heat at constant volume
            gamma : float
                the specific heat ratio
            gamma_s : float
                defined as derivative ln(P) with respect to ln(rho) at constant entropy per equation 2.71 in RP-1311
            speed_sound : float
                the speed of sound in the mixture.
        '''

        # Defines the number of moles of each species in the micture
        moles = mixture.X * (1/ mixture.mean_molecular_weight)

        # Initializes a_ij
        a_ij = np.zeros((mixture.n_elements, mixture.n_species))
        for i, element in enumerate(mixture.element_names): 
            for j, species in enumerate(mixture.species_names):
                a_ij[i,j] = mixture.n_atoms(species, element)

        # Finds specific heat at constant pressure based on equation 2.59 TODO: add terms for condensed species
        c_p =  ct.gas_constant * (
            np.sum([dpi_dlnT_P[i] * np.sum(a_ij[i,:] * moles * mixture.standard_enthalpies_RT) for i in range(mixture.n_elements)]) +
            np.sum(moles * mixture.standard_enthalpies_RT) * dlnn_dlnT_P +
            np.sum(moles * mixture.standard_cp_R) +
            np.sum(moles * mixture.standard_enthalpies_RT**2)
        )

        # Finds specifc heat at constant volume, based on equation 2.70, specific heat ratio, The isentropic exponent based on equation 2.73, 
        # and speed of sound based on equation 2.74
        c_v = c_p + mixture.P * mixture.v / mixture.T * dlnV_dlnT_P**2 / dlnV_dlnP_T
        gamma = c_p / c_v
        gamma_s = -gamma/dlnV_dlnP_T
        speed_sound = np.sqrt(ct.gas_constant * mixture.T * gamma_s/mixture.mean_molecular_weight)

        properties = {  "pressure"          : mixture.P,
                        "temperature"       : mixture.T,
                        "density"           : mixture.density_mass,
                        "specific volume"   : mixture.volume_mass,
                        "enthalpy"          : mixture.enthalpy_mass,
                        "internal energy"   : mixture.int_energy_mass,
                        "gibbs"             : mixture.gibbs_mass,
                        "entropy"           : mixture.entropy_mass,
                        "molar mass"        : mixture.mean_molecular_weight,
                        "c_p"               : c_p,
                        "c_v"               : c_v,
                        "gamma"             : gamma,                                                                            
                        "gamma_s"           : gamma_s,    
                        "speed sound"       : speed_sound          
                        }

        return properties
    
    def __chamber_properties(self, products):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic properties of said mixture, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Mixture 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        properties : dictionary
        '''

        # Equilibriates chamber returning a cantera mixture with the properties & composition at the chamber
        molar_ratio = self.of_ratio / (self.oxidizer.mean_molecular_weight / self.fuel.mean_molecular_weight)
        moles_ox = molar_ratio / (1 + molar_ratio)
        moles_f = 1 - moles_ox
        chamber_mixture = ct.Mixture([(self.fuel, moles_f), (self.oxidizer, moles_ox), (products, 0)])
        chamber_mixture.equilibrate('HP')

        # Finds thermodynamic derivatives 
        derivatives = self.__get_thermo_derivatives(products)

        # Finds thermodynamic properties
        therm_prop = self.__get_thermo_properties(products, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])

        # Calculate c* per
        char_velocity = (np.sqrt(ct.gas_constant * therm_prop["temperature"] / (therm_prop["molar mass"] * therm_prop["gamma"])) * 
                            np.power(2 / (therm_prop["gamma"] + 1), -(therm_prop["gamma"] + 1) / (2*(therm_prop["gamma"] - 1))))

        # velocity and Mach are 0 in a FAC combustor, area ratio, Isp, Ivac, and Cf are not defined at chamber
        chamber_prop = {"velocity"        : 0, 
                        "mach"            : 0, 
                        "area ratio"      : np.NaN,
                        "I_sp"            : np.NaN,
                        "I_vac"           : np.NaN,
                        "c*"              : char_velocity,
                        "C_f"             : np.NaN,
                        "mole fraction"   : products.mole_fraction_dict()}
        
        self.chamber = therm_prop | chamber_prop | derivatives

    def __throat_properties(self, products):
        '''
        Description
        --------------------------------------------------
        This is an internal method not meant for use outside the class.
        given a solution of gases & condensed species, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        None
        '''

        chamber = self.chamber
        # initial guess at throat pressure using specific heat ratio gamma
        pressure_throat = chamber["pressure"] / np.power((chamber["gamma_s"] + 1) / 2., chamber["gamma_s"] / (chamber["gamma_s"] - 1))

        # Setting up for iteration
        max_iter_throat = 10            # NOTE exceeds value of 4 from RP-1311
        tolerance_throat = 4e-5
        mach = 1.0
        num_iter = 0
        residual = 1
 
        while (residual > tolerance_throat and num_iter < max_iter_throat ) :
            num_iter += 1
           
            
            products.SP = chamber["entropy"], pressure_throat
            products.equilibrate('SP')

            throat_derivatives = self.__get_thermo_derivatives(products)
            throat_properties = self.__get_thermo_properties(products, throat_derivatives["dpi_dlnT_P"], throat_derivatives["dlnn_dlnT_P"], throat_derivatives["dlnV_dlnT_P"], throat_derivatives["dlnV_dlnP_T"])
            
            velocity = np.sqrt(2 * (chamber["enthalpy"] - throat_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * throat_properties["temperature"] * throat_properties["gamma_s"]  / throat_properties["molar mass"])
            mach = velocity / speed_sound
            
            pressure_throat = pressure_throat * (1 + throat_properties["gamma_s"] * mach**2) / (1 + throat_properties["gamma_s"] )

            residual = np.abs((velocity**2 - speed_sound**2)/velocity**2)

            print(f"pressure is: {pressure_throat:.7}\t velocity is {velocity:.4}\t speed of sound is: {speed_sound:.4}\t residual is: {np.abs((velocity**2 - speed_sound**2)/velocity**2):.4}")

        if num_iter >= max_iter_throat:
            print(f'Warning: Convergance took {num_iter} iterations which exceeds the limit of {max_iter_throat} max iterations. residual is {residual}.')
        
        # velocity and Mach are the same at throat, area ratio, Isp, Ivac, and Cf are not defined at chamber
        throat_prop = { "velocity"        : speed_sound, 
                        "mach"            : 1, 
                        "area ratio"      : np.NaN,
                        "I_sp"            : np.NaN,
                        "I_vac"           : np.NaN,
                        "c*"              : np.NaN,
                        "C_f"             : np.NaN,
                        "mole fraction"   : products.mole_fraction_dict()}

        self.throat = throat_properties | throat_prop | throat_derivatives
    
    def state_at_area(self, area_ratio, speed = "supersonic"):
        '''
        Description
        --------------------------------------------------
        Iteratively find the properties at a given location in along the nozzle given the area ratio and whether it is in the subsonic or supersonic section of the nozzle.

        Parameters
        --------------------------------------------------
        area_ratio : float
            The ratio at of the area at a given location over the area of the throat.
        speed : string
            a string with two possible values:
                'subsonic' : the converging section of the nozzle
                'supersonic' : the diverging section of the nozzle

        Returns
        --------------------------------------------------
        local_properties : Dict
            A dictionary of the thermodynamic properties and derivatives 
        
        '''

        # checking for valid input
        if area_ratio <= 1:
            raise ValueError("Area ratio was less than or equal to 1")
        
        # getting initial guess of ln(pc/pe).
        if speed == "subsonic": 
            if area_ratio > 1.000 and area_ratio < 1.09:
                lnpc_p = 0.9*np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio))
            elif area_ratio >= 1.09:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio)) 
                
        if speed == "supersonic": 
            if area_ratio > 1.000 and area_ratio < 2:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"]) +np.sqrt(3.294*np.log(area_ratio)**2+1.534*np.log(area_ratio))
            elif area_ratio >= 2:
                lnpc_p = self.chamber["gamma_s"] + 1.4 * np.log(area_ratio)
        
        # initial guess at equilibrium
        pressure = self.chamber["pressure"]/np.exp(lnpc_p)
        products = self._products
        products.SPX = self.throat["entropy"], pressure, self.throat["mole fraction"]
        products.equilibrate("SP")

        # defining iteration limits and convergence
        num_iter = 0
        max_iter = 10
        tolerance = 4e-5
        residual = 1

        # defines throat area / throat massflow
        at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])

        # iterative solver for state at position _blank_
        while residual > tolerance:
            num_iter += 1

            if num_iter == max_iter:
                print(f"exceeded {max_iter} iterations, residual is {residual}")
                break

            derivatives = self.__get_thermo_derivatives(products)
            properties = self.__get_thermo_properties(products, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2 * (self.chamber["enthalpy"] - properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * properties["temperature"] * properties["gamma_s"] / properties["molar mass"])
            a_mdot = 1/(properties["density"]*velocity)
            a_at = a_mdot/at_mdot

            dlnpc_p_dlna_at = properties["gamma_s"] * velocity**2 / (velocity**2 - speed_sound**2)
            lnpc_p = lnpc_p + dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at))
            residual = abs(dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at)))

            print(f"residual: {residual} \t\t pressure: {pressure}\t\t speed of sound: {speed_sound}")

            pressure = self.chamber["pressure"]/np.exp(lnpc_p)

            products.SP = self.throat["entropy"], pressure
            products.equilibrate('SP')

        
        local_prop  = { "velocity"        : velocity, 
                        "mach"            : velocity/speed_sound, 
                        "area ratio"      : area_ratio,
                        "I_sp"            : np.NaN,
                        "I_vac"           : np.NaN,
                        "c*"              : np.NaN,
                        "C_f"             : np.NaN,
                        "mole fraction"   : products.mole_fraction_dict()}

        local_proprties = properties | local_prop | derivatives

        return local_proprties

    def __exit_properties(self, products, exit_condition, exit_parameter='pressure'):
        '''
        This is an internal method not meant for use outside the class.
        equilbriates solution and finds thermal derivatives and properties at the exit of the engine.
      
        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        exit_condition : float
            The exit conditions of the engine. Depending on what 'exit parameter' is
            defined to be the exit condition can be defined in the following ways:
                'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
                'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
        exit_parameter : string
            A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
            or 'area ratio'. 
            
        Returns
        --------------------------------------------------
        None
        '''

        if exit_parameter == "area ratio":
            exit_properties = self.state_at_area(exit_condition, speed = "supersonic")
            ae_mdot = 1/(exit_properties["density"]*exit_properties["velocity"])

            exit_properties["I_sp"] = exit_properties["velocity"] / sp.constants.g
            exit_properties["I_vac"] = exit_properties["velocity"] / sp.constants.g + exit_properties["pressure"] * ae_mdot / sp.constants.g
            exit_properties["C_f"] = exit_properties["velocity"] / self.chamber["c*"]

            self.exit =  exit_properties
        
        elif exit_parameter == "pressure":
            pressure = exit_condition
            products.SP = self.chamber["entropy"], pressure 
            products.equilibrate('SP')

            exit_derivatives = self.__get_thermo_derivatives(products)
            exit_properties = self.__get_thermo_properties(products, exit_derivatives["dpi_dlnT_P"], exit_derivatives["dlnn_dlnT_P"], exit_derivatives["dlnV_dlnT_P"], exit_derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2* (self.chamber["enthalpy"] - exit_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * exit_properties["temperature"] * exit_properties["gamma_s"]  / exit_properties["molar mass"])
            mach = velocity / speed_sound

            at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])
            ae_mdot = 1 / (exit_properties["density"]*velocity)
            ae_at = ae_mdot/at_mdot

            print(pressure, ae_mdot, velocity, )
            Isp = velocity / sp.constants.g
            Ivac = Isp + pressure * ae_mdot / sp.constants.g
            Cf = velocity / self.chamber["c*"]

            exit_prop = {   "velocity"        : velocity, 
                            "mach"            : velocity/speed_sound, 
                            "area ratio"      : ae_at,
                            "I_sp"            : Isp,
                            "I_vac"           : Ivac,
                            "c*"              : np.NaN,
                            "C_f"             : Cf,
                            "mole fraction"   : products.mole_fraction_dict()}

            exit_properties = exit_properties | exit_prop | exit_derivatives

            self.exit = exit_properties
            
        else: 
            raise ValueError("Invalid input. exit_parameter was not defined as 'pressure' or 'area ratio'")


In [114]:
chamber_pressure = Q(600, 'psi').to_base_units().magnitude
exit_pressure = Q(14.7, 'psi').to_base_units().magnitude

example = engine_state("liquid_oxygen", "liquid_jetA", 2.2, chamber_pressure, 8, temp_oxidizer=90.17, temp_fuel=293, 
                        combustion_products=['C2H2,acetylene', 'O2', 'H2O', 'CO', 'CO2', 'H2O2', 'OH', 'H','CH4', 'C2H4', 'H2'], exit_parameter='area ratio')

pressure is: 3.199607e+06	 velocity is 901.1	 speed of sound is: 835.2	 residual is: 0.1409
pressure is: 3.141763e+06	 velocity is 817.3	 speed of sound is: 838.9	 residual is: 0.05363
pressure is: 3.162029e+06	 velocity is 845.3	 speed of sound is: 837.7	 residual is: 0.01784
pressure is: 3.155066e+06	 velocity is 835.5	 speed of sound is: 838.1	 residual is: 0.006239
pressure is: 3.157475e+06	 velocity is 838.9	 speed of sound is: 838.0	 residual is: 0.002145
pressure is: 3.156644e+06	 velocity is 837.7	 speed of sound is: 838.0	 residual is: 0.0007421
pressure is: 3.156931e+06	 velocity is 838.1	 speed of sound is: 838.0	 residual is: 0.0002561
pressure is: 3.156832e+06	 velocity is 838.0	 speed of sound is: 838.0	 residual is: 8.847e-05
pressure is: 3.156866e+06	 velocity is 838.0	 speed of sound is: 838.0	 residual is: 3.055e-05
residual: 0.31052694211529946 		 pressure: 129319.62262389521		 speed of sound: 656.694484741802
residual: 0.17918362410049513 		 pressure: 94799.11678279

             pressure  temperature   density  specific volume      enthalpy  internal energy         gibbs       entropy  molar mass          c_p          c_v     gamma   gamma_s  speed sound     velocity      mach  area ratio        I_sp       I_vac           c*       C_f                                                                                                                                                                                                                                                                                                                             mole fraction                                                    dpi_dlnT_P  dlnn_dlnT_P                                                    dpi_dlnP_T  dlnn_dlnP_T  dlnV_dlnT_P  dlnV_dlnP_T
chamber  4.136854e+06  3487.173686  3.128432         0.319649 -8.455360e+05    -2.167877e+06 -4.199721e+07  11800.866628   21.926248  4528.914400  4173.963583  1.085039  0.554186   856.050546     0.000000  0.000000      